In [51]:
!python -m pip install --upgrade pip

In [53]:
!python -m pip install -U weaviate-client==4.7.1
!python -m pip install python-dotenv==1.0.0
!python -m pip install openai==1.54.3

### Environmental Variables & Configurations

**FIRST** Create your .env file in this folder.  Use the .env.example file as a guide.

In [54]:
import os
from dotenv import load_dotenv

# Get the current file's directory (all_rag_techniques)
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Construct the path to the .env file in the parent directory
dotenv_path = os.path.join(parent_dir, '.env')

# Load the .env file
load_dotenv(dotenv_path)

print(f"Current working directory: {os.getcwd()}")
print(f"Parent directory (where .env should be): {parent_dir}")
print(f".env file path: {dotenv_path}")
print(f".env file exists: {os.path.exists(dotenv_path)}")

Current working directory: /content
Parent directory (where .env should be): /
.env file path: /.env
.env file exists: False


In [55]:
# Import Environmental Variables

#Keys and URLs for Embedding and Generative Models
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
# OPENAI_BASE_URL = os.getenv('OPENAI_BASE_URL')
OPENAI_EMBEDDING_URL = os.getenv('OPENAI_EMBEDDING_URL')
OPENAI_GENERATION_URL = os.getenv('OPENAI_GENERATION_URL')

#Keys and URLS for Vector Databases
WEAVIATE_URL_VERBA = os.getenv('WEAVIATE_URL_VERBA')
WEAVIATE_API_KEY_VERBA = os.getenv('WEAVIATE_API_KEY_VERBA')

print(len(WEAVIATE_URL_VERBA)>20)
print(len(WEAVIATE_API_KEY_VERBA)>20)
print(len(OPENAI_API_KEY)>20)

True
True
True


In [56]:
# Embedding
EMBEDDING_MODEL = "text-embedding-3-small"
ENCODING_FORMAT = "float"

# Database
COLLECTION_NAME = "VERBA_Embedding_text_embedding_3_small"

### Vector Database Connection

In [57]:
# VECTOR DATABASE CONNECTION

from weaviate.classes.init import Auth, AdditionalConfig, Timeout
import weaviate

client_db = weaviate.connect_to_weaviate_cloud(
    cluster_url = WEAVIATE_URL_VERBA,
    auth_credentials = Auth.api_key(WEAVIATE_API_KEY_VERBA),
    additional_config=AdditionalConfig(timeout=Timeout(init=30, query=60, insert=120))  # Values in seconds
)

print(client_db.is_ready())

True


### Embedding Service Connection

In [58]:
# Extracts the actual vector embedding from the OpenAI response
# https://platform.openai.com/docs/api-reference/embeddings/create
def openai_extract_vector(
        response
    ) -> list[float]:

    return response.data[0].embedding

In [59]:
# EMBEDDING CONNECTION - OPENAI
# https://platform.openai.com/docs/api-reference/authentication

import openai
from openai import OpenAI

# Set API Key.  Not necessary if you have an
# OPENAI_API_KEY variable in your environment
openai.api_key = OPENAI_API_KEY

client_embedding = OpenAI()

In [60]:
#print(type(client_embedding))

#for item in client_embedding.models.list():
#    print(item)

### Generation Service Connection

In [61]:
# GENERATION CONNECTION - OPENAI
# https://platform.openai.com/docs/api-reference/authentication

import openai
from openai import OpenAI

# Set API Key.  Not necessary if you have an
# OPENAI_API_KEY variable in your environment
openai.api_key = OPENAI_API_KEY

generation_client = OpenAI()

### Simple RAG

In [62]:
from weaviate.classes.query import MetadataQuery

query_text = "I want the no of credit complaints that were registered in January 2024 by Wells Fargo"

# Vectorize the query
response_embedding = client_embedding.embeddings.create(
    model = EMBEDDING_MODEL,
    input = query_text,
    encoding_format = ENCODING_FORMAT
)

# Extract the verctor embeddings list[float] from the embedding response
query_vector = openai_extract_vector(response_embedding)

# Look up the appropriate Weviate database collection - name based on embedding model used
collection = client_db.collections.get('VERBA_Embedding_text_embedding_3_small')

# Send vector query to database and get response
db_response = collection.query.near_vector(
    near_vector=query_vector,
    limit=10,
    return_metadata=MetadataQuery(distance=True)
)

# Extract items from database response
# and aggregate into a single string
db_response_content = ""
for item in db_response.objects:
    segment = '\n<ContextSegment' + str(int(item.properties.get('chunk_id'))) + '>\n'
    db_response_content += segment
    db_response_content += item.properties.get('content')

# # Print results
# for item in response_db.objects:
#     print(item.properties)
#     print(item.metadata.distance)

# print(response_content)

In [63]:
# Response Generation with OpenAI
# # https://platform.openai.com/docs/api-reference/chat/create


generation_response = generation_client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": f"You are a helpful assistant who uses this context if appropriate: {db_response_content}"
    },
    {
      "role": "user",
      "content": query_text
    }
  ]
)

print(generation_response.choices[0].message.content)

The data provided does not specify the exact number of credit complaints registered by Wells Fargo in January 2024. However, based on the data segments, there is at least one credit complaint related to a credit card issue by Wells Fargo logged on 01/25/24, as seen in <ContextSegment624>. Further specific details would be needed to provide an exact number.


### Unit Testing

In [69]:
from weaviate.classes.query import MetadataQuery

# List of test queries
test_queries = [
    "What are the top complaint categories for Wells Fargo?",
    "How many complaints were filed in June 2024 for Wells Fargo?"
]

# Define expected outputs for each query
expected_outputs = [
    "The top complaint categories for Wells Fargo include Checking account, General-purpose credit card or charge card and Credit reporting.",
    "There were 64 complaints filed in June 2024 for Wells Fargo."
]

# Validate results
for idx, query_text in enumerate(test_queries, 1):
    print(f"\n### Test Query {idx}: {query_text} ###")

    # Generate vector embeddings for the query
    response_embedding = client_embedding.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query_text,
        encoding_format=ENCODING_FORMAT
    )

    # Extract vector embeddings as a list of floats
    query_vector = openai_extract_vector(response_embedding)

    # Look up the appropriate Weaviate database collection - name based on embedding model used
    collection = client_db.collections.get('VERBA_Embedding_text_embedding_3_small')

    # Send vector query to the database and get response
    db_response = collection.query.near_vector(
        near_vector=query_vector,
        limit=10,
        return_metadata=MetadataQuery(distance=True)
    )

    # Extract and aggregate items from the database response
    db_response_content = ""
    for item in db_response.objects:
        segment = '\n<ContextSegment ' + str(int(item.properties.get('chunk_id'))) + '>\n'
        db_response_content += segment
        db_response_content += item.properties.get('content')

    # Generate a response using the retrieved context
    generation_response = generation_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": f"You are a helpful assistant who uses this context if appropriate: {db_response_content}"
            },
            {
                "role": "user",
                "content": query_text
            }
        ]
    )

    # Extract the generated response
    generated_response = generation_response.choices[0].message.content

    # Compare with the expected output
    expected_output = expected_outputs[idx - 1]

    print("\n### Expected vs Generated Response ###")
    print(f"Expected: {expected_output}")
    print(f"Generated: {generated_response}")

    if generated_response.strip() == expected_output.strip():
        print("✅ Test Passed")
    else:
        print("❌ Test Failed")
        print("Difference:")
        print(f"Expected: {expected_output.strip()}")
        print(f"Generated: {generated_response.strip()}")



### Test Query 1: What are the top complaint categories for Wells Fargo? ###

### Expected vs Generated Response ###
Expected: The top complaint categories for Wells Fargo include Checking account, General-purpose credit card or charge card and Credit reporting.
Generated: The top complaint categories for Wells Fargo include Checking account, General-purpose credit card or charge card, and Credit reporting.
❌ Test Failed
Difference:
Expected: The top complaint categories for Wells Fargo include Checking account, General-purpose credit card or charge card and Credit reporting.
Generated: The top complaint categories for Wells Fargo include Checking account, General-purpose credit card or charge card, and Credit reporting.

### Test Query 2: How many complaints were filed in June 2024 for Wells Fargo? ###

### Expected vs Generated Response ###
Expected: There were 64 complaints filed in June 2024 for Wells Fargo.
Generated: There were 64 complaints filed in June 2024 for Wells Fargo.
✅